# Convolutional Neural Network (CNN)

CNN on MNIST data-set using TensorFlow.

In [1]:
# Import libraries

import numpy as np
import tensorflow as tf

In [2]:
tf.__version__

'1.1.0'

In [3]:
# Load the data-set
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets("data/MNIST/", one_hot=True)

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


In [4]:
# Insight on the data downloaded
print("Size of: ")
print("- Training-set:\t\t{}".format(len(data.train.labels)))
print("- Test-set:\t\t{}".format(len(data.test.labels)))
print("- Validation-set:\t{}".format(len(data.validation.labels)))

Size of: 
- Training-set:		55000
- Test-set:		10000
- Validation-set:	5000


In [6]:
# Data dimensions
img_size = 28
img_size_flat = img_size * img_size
img_shape = (img_size, img_size)
num_classes = 10

In [7]:
# Configuration of neural network

# Convolution layer 1
# Convolution filters of size 5X5
# 16 of these filters
filter_size1 = 5
num_filters1 = 16

# Convolution layer 2
# Convolution filter size 5X5
# 32 of these filters
filter_size2 = 5
num_filters2 = 32

# Fully connected layer
# # of neurons in fc layer
fc_size = 128

In [8]:
# Helper functions

# new weights drawn from normal distribution with stddev = 0.05
def weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

# new biases
def biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

# convolution operation
def conv2d(X, w):
    return tf.nn.conv2d(X, w, strides=[1,1,1,1], padding='SAME')

# max-pooling or down sampling
def max_pool(X):
    return tf.nn.max_pool(X, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

# flatten 4D tensor to 2D tensor
def flatten(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:].num_elements()
    return tf.reshape(layer, [-1, num_features]), num_features

In [9]:
# Placeholder variables
X = tf.placeholder(tf.float32, [None, img_size_flat])
y_true = tf.placeholder(tf.float32, [None, num_classes])
y_true_class = tf.argmax(y_true, dimension=1)

In [10]:
# 1st convolution layer

# Reshape input image to a 4D tensor [num_imges, width, height, channel]
X_img = tf.reshape(X, [-1, img_size, img_size, 1])
# weight [filter_size, filter_size, num_input_channels, num_filters]
w1 = weights([filter_size1, filter_size1, 1, num_filters1])
b1 = biases(num_filters1)
h_conv1 = tf.nn.relu(conv2d(X_img, w1) + b1)
h_pool1 = max_pool(h_conv1)

In [11]:
h_pool1

<tf.Tensor 'MaxPool:0' shape=(?, 14, 14, 16) dtype=float32>

In [12]:
# 2nd convolution layer

w2 = weights([filter_size2, filter_size2, 16, num_filters2])
b2 = biases(num_filters2)
h_conv2 = tf.nn.relu(conv2d(h_pool1, w2) + b2)
h_pool2 = max_pool(h_conv2)

In [13]:
h_pool2

<tf.Tensor 'MaxPool_1:0' shape=(?, 7, 7, 32) dtype=float32>

In [14]:
# flatten h_pool2
layer_flat, num_features = flatten(h_pool2)

In [15]:
layer_flat

<tf.Tensor 'Reshape_1:0' shape=(?, 1568) dtype=float32>

In [16]:
num_features

1568

In [17]:
# Fully connected (FC) layer 1
w_fc1 = weights([num_features, fc_size])
b_fc1 = biases(fc_size)
h_fc1 = tf.nn.relu(tf.matmul(layer_flat, w_fc1) + b_fc1)

In [18]:
h_fc1

<tf.Tensor 'Relu_2:0' shape=(?, 128) dtype=float32>

In [19]:
# output layer
w_fc2 = weights([fc_size, num_classes])
b_fc2 = biases(num_classes)
h_fc2 = tf.matmul(h_fc1, w_fc2) + b_fc2

In [20]:
h_fc2

<tf.Tensor 'add_3:0' shape=(?, 10) dtype=float32>

In [21]:
y_pred = tf.nn.softmax(h_fc2)

In [22]:
y_pred_class = tf.argmax(y_pred, dimension=1)

In [23]:
# cross entropy
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=h_fc2, labels=y_true)

In [24]:
cost = tf.reduce_mean(cross_entropy)

In [25]:
# optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

In [26]:
correct_prediction = tf.equal(y_pred_class, y_true_class)

In [27]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [28]:
# Initialize tensorflow
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [29]:
# batch size for sgd
batch_size = 128

In [30]:
from tqdm import tqdm
total_iter = 0
def optimize(num_iter):
    global total_iter
    for i in tqdm(range(total_iter, total_iter + num_iter)):
        batch_X, batch_y = data.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={X:batch_X, y_true:batch_y})

In [31]:
def print_accuracy():
    acc = accuracy.eval(session=sess, feed_dict={X: data.validation.images,
                                        y_true: data.validation.labels})
    print('Accuracy on Validation = {:.1%}'.format(acc))

In [32]:
print_accuracy()

Accuracy = 10.8%


In [33]:
optimize(1)

100%|██████████| 1/1 [00:00<00:00,  1.36it/s]


In [34]:
print_accuracy()

Accuracy = 12.4%


In [35]:
optimize(99)

100%|██████████| 99/99 [00:52<00:00,  1.88it/s]


In [36]:
print_accuracy()

Accuracy = 65.8%


In [37]:
optimize(900)

100%|██████████| 900/900 [08:04<00:00,  1.84it/s]


In [38]:
print_accuracy()

Accuracy = 94.1%


In [39]:
sess.close()